In [1]:
import pandas as pd
import requests
from requests.exceptions import InvalidURL
import json
import ast
import math
import urllib.request
import json


In [2]:
df = pd.read_excel("controlled_substances.xlsx")

In [3]:
df

,SUBSTANCE,CSCN,SCH,NARC,OTHER NAMES
0,"(1-(4-Fluorobenzyl)-1H-indol-3-yl)(2,2,3,3- te...",7014.0,I,N,FUB-144
1,"[3,2-c]-furazan-5α-androstan-17β-ol",4000.0,III,N,NaN
2,"[3,2-c]pyrazole-androst-4-en-17β-ol",4000.0,III,N,NaN
3,1-(1-Phenylcyclohexyl)pyrrolidine,7458.0,I,N,"PCPy, PHP, rolicyclidine"
4,1-(2-Phenylethyl)-4-phenyl-4-acetoxypiperidine,9663.0,I,Y,"PEPAP, synthetic heroin"
...,...,...,...,...,...
550,Zipeprol (1-methoxy-3[-4-(2-methoxy-2- phenyle...,9873.0,I,Y,NaN
551,Zolpidem,2783.0,IV,N,"Ambien, Ivadal, Stilnoct, Stilnox"
552,Zopiclone,2784.0,IV,N,Lunesta
553,Zuranolone,2420.0,IV,N,ZURZUVAE


In [4]:
df = df.drop(df[df["NARC"]=="N"].index).reset_index(drop=True)

In [5]:
df

,SUBSTANCE,CSCN,SCH,NARC,OTHER NAMES
0,1-(2-Phenylethyl)-4-phenyl-4-acetoxypiperidine,9663.0,I,Y,"PEPAP, synthetic heroin"
1,1-Methyl-4-phenyl-4-propionoxypiperidine,9661.0,I,Y,"MPPP, synthetic heroin"
2,2'-Fluoro ortho-fluorofentanyl (N-(1-(2- fluor...,9855.0,I,Y,NaN
3,2-methoxy-N-(1-phenethylpiperidin-4-yl)-N- phe...,9825.0,I,Y,Methoxyacetyl fentanyl
4,3-Methylfentanyl,9813.0,I,Y,"China White, fentanyl"
...,...,...,...,...,...
192,Tramadol (2-[(dimethylamino)methyl]-1-(3- meth...,9752.0,IV,Y,Tramadol
193,Trimeperidine,9646.0,I,Y,Promedolum
194,"U-47700 (3,4-dichloro-N-[2-(dimethylamino)cycl...",9547.0,I,Y,U-47700
195,Valeryl fentanyl,9840.0,I,Y,N-(1-phenethylpiperidin-4-yl)-N-phenylpentanamide


In [6]:
# Function to get the rxcui for a single ingredient
def get_single_rxcui(item):
    item_encoded = urllib.parse.quote(item)
    url = f"https://rxnav.nlm.nih.gov/REST/rxcui.json?name={item_encoded}&search=2"

    try:
        with urllib.request.urlopen(url) as response:
            data = json.loads(response.read())
            rxcui = data.get('idGroup', {}).get('rxnormId')
            if rxcui:
                return ",".join(rxcui)
    except Exception as e:
        print(f"Error for ingredient {item}: {e}")

    return None

# Function to process an entry in the 'OTHER NAMES' column
def get_rxcui_for_other_names(entry):
    if pd.isna(entry):
        return None

    ingredients = entry.split(',')  # Split by comma for multiple ingredients
    rxcui_list = [get_single_rxcui(ingredient.strip()) for ingredient in ingredients]
    rxcui_list = [rxcui for rxcui in rxcui_list if rxcui is not None]  # Filter out None values

    return ','.join(rxcui_list) if rxcui_list else None

# Apply the function to each row in the DataFrame for the 'SUBSTANCE' and 'OTHER NAMES' column
df['rxcui_SUBSTANCE'] = df['SUBSTANCE'].apply(get_single_rxcui)
df['rxcui_OTHER_NAMES'] = df['OTHER NAMES'].apply(get_rxcui_for_other_names)


Error for ingredient : HTTP Error 400: Bad Request


In [8]:
df

,SUBSTANCE,CSCN,SCH,NARC,OTHER NAMES,rxcui_SUBSTANCE,rxcui_OTHER_NAMES
0,1-(2-Phenylethyl)-4-phenyl-4-acetoxypiperidine,9663.0,I,Y,"PEPAP, synthetic heroin",None,None
1,1-Methyl-4-phenyl-4-propionoxypiperidine,9661.0,I,Y,"MPPP, synthetic heroin",None,None
2,2'-Fluoro ortho-fluorofentanyl (N-(1-(2- fluor...,9855.0,I,Y,NaN,None,None
3,2-methoxy-N-(1-phenethylpiperidin-4-yl)-N- phe...,9825.0,I,Y,Methoxyacetyl fentanyl,None,None
4,3-Methylfentanyl,9813.0,I,Y,"China White, fentanyl",None,4337
...,...,...,...,...,...,...,...
192,Tramadol (2-[(dimethylamino)methyl]-1-(3- meth...,9752.0,IV,Y,Tramadol,None,10689
193,Trimeperidine,9646.0,I,Y,Promedolum,None,None
194,"U-47700 (3,4-dichloro-N-[2-(dimethylamino)cycl...",9547.0,I,Y,U-47700,None,None
195,Valeryl fentanyl,9840.0,I,Y,N-(1-phenethylpiperidin-4-yl)-N-phenylpentanamide,None,None


In [12]:
# Remove rows where both 'rxcui_SUBSTANCE' and 'rxcui_OTHER_NAMES' are None or empty
df = df.dropna(subset=['rxcui_SUBSTANCE', 'rxcui_OTHER_NAMES'], how='all')


In [13]:
df.reset_index(drop=True, inplace=True)

In [14]:
df.to_excel("controlled_substances_opioids.xlsx", index=False)

In [21]:
df_og = pd.read_excel("opioid_ingredients.xlsx")

In [22]:
df_og

,Ingredient,Physician
0,alfentanil,NaN
1,alphaprodine,NaN
2,Anileridine,PH
3,benzhydrocodone,NaN
4,buprenorphine,NaN
5,butorphanol,NaN
6,codeine,NaN
7,dextromoramide,NaN
8,dezocine,NaN
9,difenoxin,PH


In [25]:
df

,SUBSTANCE,CSCN,SCH,NARC,OTHER NAMES,rxcui_SUBSTANCE,rxcui_OTHER_NAMES
0,3-Methylfentanyl,9813.0,I,Y,"China White, fentanyl",None,4337
1,3-Methylthiofentanyl,9833.0,I,Y,"China White, fentanyl",None,4337
2,Acetylmethadol,9601.0,I,Y,Methadyl acetate,6814,6814
3,Alfentanil,9737.0,II,Y,Alfenta,480,None
4,Alpha-methylfentanyl,9814.0,I,Y,"China White, fentanyl",None,4337
5,Alpha-methylthiofentanyl,9832.0,I,Y,"China White, fentanyl",None,4337
6,Alphaprodine,9010.0,II,Y,Nisentil,594,None
7,Anileridine,9020.0,II,Y,Leritine,17933,None
8,Beta-hydroxy-3-methylfentanyl,9831.0,I,Y,"China White, fentanyl",None,4337
9,Beta-hydroxyfentanyl,9830.0,I,Y,"China White, fentanyl",None,4337


In [24]:
df_og['rxcui'] = df_og["Ingredient"].apply(get_single_rxcui)
df_og

,Ingredient,Physician,rxcui
0,alfentanil,NaN,480
1,alphaprodine,NaN,594
2,Anileridine,PH,17933
3,benzhydrocodone,NaN,2001352
4,buprenorphine,NaN,1819
5,butorphanol,NaN,1841
6,codeine,NaN,2670
7,dextromoramide,NaN,3290
8,dezocine,NaN,22713
9,difenoxin,PH,23024


In [26]:
# Assuming df has columns 'rxcui_SUBSTANCE' and 'rxcui_OTHER_NAMES'
# and df_og has column 'rxcui'

# Convert all RxCUIs in df to a set for faster search
rxcui_set_df = set()
for rxcuis in df['rxcui_SUBSTANCE'].dropna().str.split(','):
    rxcui_set_df.update(rxcuis)
for rxcuis in df['rxcui_OTHER_NAMES'].dropna().str.split(','):
    rxcui_set_df.update(rxcuis)

# Convert all RxCUIs in df_og to a set for faster search
rxcui_set_df_og = set(df_og['rxcui'].dropna().unique())

# Check if RxCUIs in df_og are in df
df_og['in_df'] = df_og['rxcui'].apply(lambda x: x in rxcui_set_df)

# Check if RxCUIs in df are in df_og
df['in_df_og_SUBSTANCE'] = df['rxcui_SUBSTANCE'].apply(lambda x: any(item in rxcui_set_df_og for item in str(x).split(',')))
df['in_df_og_OTHER_NAMES'] = df['rxcui_OTHER_NAMES'].apply(lambda x: any(item in rxcui_set_df_og for item in str(x).split(',')))

# Combine the two 'in_df_og' checks into one column for easier handling
df['in_df_og'] = df['in_df_og_SUBSTANCE'] | df['in_df_og_OTHER_NAMES']

# Create a DataFrame for all unique RxCUIs
all_rxcui_df = pd.DataFrame({'rxcui': list(rxcui_set_df)})
all_rxcui_df_og = pd.DataFrame({'rxcui': list(rxcui_set_df_og)})

# Combine both DataFrames, removing duplicates
combined_rxcui_df = pd.concat([all_rxcui_df, all_rxcui_df_og]).drop_duplicates().reset_index(drop=True)


In [27]:
combined_rxcui_df

,rxcui
0,594
1,215008
2,216903
3,3304
4,28874
...,...
74,6377
75,7676
76,8001
77,2001352


In [30]:
df_og

,Ingredient,Physician,rxcui,in_df
0,alfentanil,NaN,480,True
1,alphaprodine,NaN,594,True
2,Anileridine,PH,17933,True
3,benzhydrocodone,NaN,2001352,False
4,buprenorphine,NaN,1819,True
5,butorphanol,NaN,1841,False
6,codeine,NaN,2670,True
7,dextromoramide,NaN,3290,True
8,dezocine,NaN,22713,False
9,difenoxin,PH,23024,True


In [31]:
df_og = df_og[df_og['in_df']==False]

In [32]:
df_og

,Ingredient,Physician,rxcui,in_df
3,benzhydrocodone,NaN,2001352,False
5,butorphanol,NaN,1841,False
8,dezocine,NaN,22713,False
18,levomethadone,PH,236913,False
20,levopropoxyphene,PH,6377,False
23,meptazinol,NaN,6761,False
27,nalbuphine,NaN,7238,False
29,noscapine,PH,7533,False
30,oliceridine,NaN,2392230,False
31,opium,NaN,7676,False


In [36]:
df_og.to_excel("CS1.xlsx", index=False)

In [34]:
df_unmatch = df[df['in_df_og']==False]

In [37]:
df_unmatch.to_excel("CS2.xlsx", index=False)